In [16]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, activations, losses, metrics, optimizers
import numpy as np

In [17]:
def create_model(input_shape=(224,224,3), num_of_classes = 1000):
    inputs = tf.keras.Input(shape=input_shape)
    x = inputs
    if input_shape[0] > 200:
        x = layers.Conv2D(96, 11, strides=(4,4), activation="relu")(x)
        x = layers.BatchNormalization()(x)
        x = layers.MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='same')(x)
    x = layers.Conv2D(256, 5, activation="relu")(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='same')(x)
    x = layers.Conv2D(384, 3, activation="relu")(x)
    x = layers.Conv2D(384, 3, activation="relu")(x)
    x = layers.Conv2D(256, 3, activation="relu")(x)
    x = layers.MaxPooling2D(pool_size=(3,3), strides=(2,2))(x)
    x = layers.Flatten()(x)
    x = layers.Dense(4096, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(4096, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    predictions = layers.Dense(num_of_classes, activation='softmax')(x)
    model = tf.keras.Model(inputs=inputs, outputs=predictions)
    return model

# model = create_model((224,224,3)) # for imagenet
model = create_model((32,32,3), 100) # for cifar100
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 28, 28, 256)       19456     
_________________________________________________________________
batch_normalization_1 (Batch (None, 28, 28, 256)       1024      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 14, 14, 256)       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 12, 12, 384)       885120    
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 10, 10, 384)       1327488   
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 8, 8, 256)         8849

In [18]:
model.compile(
#     loss=keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    loss = keras.losses.CategoricalCrossentropy(from_logits=False),
    optimizer = keras.optimizers.SGD(learning_rate=0.01, momentum=0.9),
    metrics=["accuracy"],
)


In [19]:
import tensorflow_datasets as tfds
import cv2
from tensorflow.python.keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split
# class Globals:
#     IMAGENET_DATA_DIR = '../../../AIACADEMY/lesson_16/data_imagenet_v2'

# dataset_test_raw,test_info = tfds.load('imagenet_v2', download=True,
#                                   data_dir = Globals.IMAGENET_DATA_DIR,
#                                   split='test[50%:]',shuffle_files=True,
#                                   as_supervised=True,with_info=True)

# def dataset_take_to_xy_np_arrays(ds):
#     images, labels = [], []
#     for image, label in ds:
#         image = cv2.resize(tfds.as_numpy(image), (224,224), interpolation = cv2.INTER_AREA)
#         image = tf.cast(image, tf.float32)
# #         image = image[np.newaxis, :]
#         images.append(image)
#         label = to_categorical(label, num_classes=1000)
#         labels.append(label)
#     return np.array(images), np.array(labels)


# X, y = dataset_take_to_xy_np_arrays(dataset_test_raw.take(-1))

# x_train, x_test, y_train, y_test = train_test_split( X, y, test_size=0.1, random_state=42)
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar100.load_data()

y_train = to_categorical(y_train, num_classes=100)
y_test = to_categorical(y_test, num_classes=100)


x_train.shape, y_train.shape, x_test.shape, y_test.shape


((50000, 32, 32, 3), (50000, 100), (10000, 32, 32, 3), (10000, 100))

In [20]:
datagen = keras.preprocessing.image.ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.4,
    height_shift_range=0.4,
    vertical_flip=True,
    horizontal_flip=True,
    validation_split=0.2)
# compute quantities required for featurewise normalization
# (std, mean, and principal components if ZCA whitening is applied)
datagen.fit(x_train)


In [ ]:
# history = model.fit(x_train, y_train, batch_size=128, epochs=100, validation_split=0.3)

# fits the model on batches with real-time data augmentation:
history = model.fit(datagen.flow(x_train, y_train, batch_size=128,
         subset='training'),
         validation_data=datagen.flow(x_train, y_train, batch_size=32, subset='validation'),
         epochs=20)

test_scores = model.evaluate(x_test, y_test)
print("Test loss:", test_scores[0])
print("Test accuracy:", test_scores[1])# summarize history for accuracy

Epoch 1/20
313/313 [==============================] - 34s 108ms/step - loss: 4.3864 - accuracy: 0.0364 - val_loss: 4.2202 - val_accuracy: 0.0594
Epoch 2/20
313/313 [==============================] - 33s 106ms/step - loss: 4.1514 - accuracy: 0.0604 - val_loss: 4.0216 - val_accuracy: 0.0772
Epoch 3/20
313/313 [==============================] - 33s 107ms/step - loss: 4.0383 - accuracy: 0.0800 - val_loss: 3.9375 - val_accuracy: 0.0894
Epoch 4/20
313/313 [==============================] - 33s 107ms/step - loss: 3.9357 - accuracy: 0.0925 - val_loss: 3.8430 - val_accuracy: 0.1065
Epoch 5/20
313/313 [==============================] - 34s 108ms/step - loss: 3.8563 - accuracy: 0.1028 - val_loss: 3.7877 - val_accuracy: 0.1150
Epoch 6/20
313/313 [==============================] - 33s 106ms/step - loss: 3.7898 - accuracy: 0.1160 - val_loss: 3.6860 - val_accuracy: 0.1353
Epoch 7/20
313/313 [==============================] - 33s 106ms/step - loss: 3.7203 - accuracy: 0.1255 - val_loss: 3.6613 - val_ac

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()